In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
df=pd.read_csv('../input/strain/strain.csv')
ks=df

df


In [ ]:
#plotdata
plt.figure(figsize=(6,6))
plt.title('q-es')
plt.plot(df['es'],df['q'])
plt.xlabel('es', fontsize=18)
plt.ylabel('q', fontsize=18)
plt.show()

plt.figure(figsize=(6,6))
plt.title('q-es')
plt.plot(df['p'],df['q'])
plt.xlabel('p', fontsize=18)
plt.ylabel('q', fontsize=18)
plt.show()

In [ ]:
ks

In [ ]:
ks['q']

In [ ]:
#setup train data
ks['esn']=ks.iloc[1:-1,2].reset_index(drop=True)
ks['qn']=ks.iloc[1:-1,0].reset_index(drop=True)
ks['pn']=ks.iloc[1:-1,1].reset_index(drop=True)
xi=ks.drop(['es','qn','pn','u'], axis=1)
#yi=ks[['qn']].copy()
yi=ks[['qn','pn']].copy()
xi=xi.dropna()
yi=yi.dropna()

x_train1=xi.to_numpy()
y_train1=yi.to_numpy()


In [ ]:
ks

In [ ]:
xi

In [ ]:
yi

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
test_split=0.3
scaler = MinMaxScaler(feature_range=(-1,1))
scaler2 = MinMaxScaler(feature_range=(-1,1))
#scaler=StandardScaler()
#scaler2=StandardScaler()
scaled_data1 = scaler.fit_transform(x_train1)
scaled_data2 = scaler2.fit_transform(y_train1)
y=scaled_data2
x=scaled_data1
x_trainn, x_test, y_train, y_test = train_test_split(x, y, test_size=test_split, random_state=42)
x_train= np.reshape(x_trainn, (x_trainn.shape[0], 1,x_trainn.shape[1]))
x_test= np.reshape(x_test, (x_test.shape[0], 1,x_test.shape[1]))


In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from matplotlib import pyplot as plt


#Build the LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape= (x_train.shape[1],x_train.shape[2])))
#model.add(LSTM(50, return_sequences= False))
model.add(Dense(100))
model.add(Dense(100))


model.add(Dense(y_train.shape[1]))
# Compile the model
model.compile(optimizer='Adam', loss='mean_squared_error',metrics=['accuracy'])

#Train the model
history=model.fit(x_train, y_train, batch_size=1,validation_data=(x_test, y_test), verbose=1,epochs=10)

# plot train and validation loss
#plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.save('strain.h5')

In [ ]:
#predict next days

from tensorflow import keras
from pandas_datareader.data import DataReader
model = keras.models.load_model('strain.h5')
output=pd.DataFrame()
predict=1000
strain_increment=0.01
ks=df
ini=np.array([1,50,.01]) #initial [q,p,strain]
ini=np.reshape(ini,(1,3))


for i in range(predict):
    ini_scaled = scaler.transform(ini)
    ini_model=np.reshape(ini_scaled,(1,1,3))
    predict = model.predict(ini_model)
    predict=np.reshape(predict,(1,predict.shape[2]))
    pre_nonscale = scaler2.inverse_transform(predict)
    #output = output.append(({'q': pre_nonscale[0,0], 'es':ini[0,2]}),ignore_index=True)
    output = output.append(({'q': pre_nonscale[0,0],'p':pre_nonscale[0,1], 'es':ini[0,2]}),ignore_index=True)
    ini[0,2]=ini[0,2]+strain_increment
    ini[0,0]=pre_nonscale[0,0]
    ini[0,1]=pre_nonscale[0,1]



#plotdata
plt.figure(figsize=(16,8))
plt.title('stress-strain')
plt.plot(output['es'],output['q'])
plt.plot(df['es'],df['q'])
plt.xlabel('es', fontsize=18)
plt.ylabel('q', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
plt.title('stress-strain')
plt.plot(output['p'],output['q'])
plt.plot(df['p'],df['q'])
plt.xlabel('p', fontsize=18)
plt.ylabel('q', fontsize=18)
plt.show()